<a href="https://colab.research.google.com/github/hsswkwk/turbo-chainsaw/blob/feature-add-object-detection/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 物体検出（Object detection）
画像や動画の中から特定の物体を検出し、その位置を特定する技術

<br>

## 物体検出のアーキテクチャ
### 2段階検出器（Two-stage detectors）
まず候補領域を抽出し、次にその候補領域に対して分類と位置の特定を行うという2段階の処理を行う手法。精度は高いが処理速度は遅い。

#### Faster R-CNN
領域提案ネットワーク (Region Proposal Network: RPN)で候補領域を抽出し、Fast R-CNNによって物体の分類、位置の予測を行う。

#### Mask R-CNN
Faster R-CNNの拡張。物体のマスクを予測するマスクブランチをネットワークに追加し、Rolプーリングの代わりにRolAlignを採用している。

<br>

### 1段階検出器（One-stage detectors）
候補領域の抽出と分類・位置の特定を同時に行う手法。処理速度は速いが精度は低い。

#### YOLO（You Only Look Once）
画像全体を一度だけ見て、物体の検出を行う。以下のステップで処理を行う。
1. 画像をグリッドに分割
2. 各グリッドセルでバウンディングボックスとクラス確率を予測
3. Non-Maximum Suppression (NMS) で重複するバウンディングボックスを削除

#### SSD（Single Shot Multibox Detector）
YOLOとは異なるアプローチの物体検出。以下の特徴を持つ。
1. 複数スケールの特徴マップを使用
2. デフォルトボックス (アンカーボックス)を基準にして、物体の位置とクラスを予測
3. 畳み込み層でバウンディングボックスとクラス確率を予測
4. Non-Maximum Suppression (NMS)で最も確度の高いバウンディングボックスを残す

#### RetinaNet
Focal Lossという損失関数を導入して1段階検出器の精度を向上させた。以下の特徴を持つ。
1. Feature Pyramid Network (FPN)で異なる大きさの物体を検出
2. Focal Lossでクラスの不均衡問題の対処を行う


In [3]:
!pip install tensorflow==2.18.0
!pip install opencv-python
!pip install matplotlib
!pip install pyyaml
!pip install protobuf==3.20.3
!if [ -d models ]; then rm -rf models; fi
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 99373, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 99373 (delta 99), reused 55 (delta 48), pack-reused 99228 (from 6)
Receiving objects: 100% (99373/99373), 623.32 MiB | 27.24 MiB/s, done.
Resolving deltas: 100% (72186/72186), done.


In [4]:
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.builders import model_builder
from object_detection.utils import label_map_util
import cv2
import numpy as np
from object_detection.utils import visualization_utils as viz_utils
from matplotlib import pyplot as plt

AttributeError: module 'keras._tf_keras.keras.layers' has no attribute 'experimental'

In [1]:
# モデルの構成ファイルのパス
CONFIG_PATH = 'models/research/object_detection/configs/tf2/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config'

# モデルを構築
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# チェックポイントのパス
CKPT_PATH = 'models/research/object_detection/test_data/checkpoint/ckpt-0'
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(CKPT_PATH)

# ラベルマップのパス
LABEL_PATH = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(LABEL_PATH)

# 画像のパス
IMAGE_PATH = 'models/research/object_detection/test_data/image1.jpg'
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

# 推論の実行
input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detection_model(input_tensor)

# 結果の描画
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'] + label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.30,
        agnostic_mode=False
)

plt.figure(figsize=(12, 16))
plt.imshow(image_np_with_detections)
plt.show()


NameError: name 'config_util' is not defined